In [ ]:
import tensorflow
import numpy as np
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import os
import zipfile

import datetime




In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [ ]:
!unzip -q "/content/drive/MyDrive/data.zip"


In [ ]:
ydir1 = '/content/content/sign'
dir2 = '/content/train/sign'
dir3 = '/content/valid/sign'

dir_list = os.listdir(dir1)
dir_list.sort()

for img in dir_list[:25001]:
  shutil.move(os.path.join(dir1,img), os.path.join(dir2,img))

for img in dir_list[25001:]:
  shutil.move(os.path.join(dir1,img), os.path.join(dir3,img))

In [ ]:
dir1 = '/content/content/unsign'
dir2 = '/content/train/unsign'
dir3 = '/content/valid/unsign'

dir_list = os.listdir(dir1)
dir_list.sort()

for img in dir_list[:25001]:
  shutil.move(os.path.join(dir1,img), os.path.join(dir2,img))

for img in dir_list[25001:]:
  shutil.move(os.path.join(dir1,img), os.path.join(dir3,img))

In [ ]:
!rm -r "/content/valid/.ipynb_checkpoints"
!rm -r "/content/train/.ipynb_checkpoints"

In [ ]:
train = ImageDataGenerator(rescale=1./255).flow_from_directory(directory="/content/train", batch_size=32, target_size=(256,256), class_mode='binary')
valid = ImageDataGenerator(rescale=1./255).flow_from_directory(directory="/content/val", batch_size=32, target_size=(256,256), class_mode='binary')

Found 46978 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
model = Sequential()
model.add(layers.Conv2D(input_shape=(256, 256, 3), filters=64,kernel_size=(3,3), padding="same",activation='relu'))
model.add(layers.Conv2D(filters=32,kernel_size=(3,3), padding="same",activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                patience=3)

In [ ]:
model_fit = model.fit(train, steps_per_epoch = 100,
                     epochs = 20,
                    verbose = 1,
                     validation_data = valid,
                      callbacks = early_stopping)

Epoch 1/20
100/100 [==============================] - 39s 239ms/step - loss: 0.5904 - accuracy: 0.7059 - val_loss: 0.4997 - val_accuracy: 0.7975
Epoch 2/20
100/100 [==============================] - 23s 228ms/step - loss: 0.4336 - accuracy: 0.8228 - val_loss: 0.4349 - val_accuracy: 0.8170
Epoch 3/20
100/100 [==============================] - 24s 236ms/step - loss: 0.4122 - accuracy: 0.8281 - val_loss: 0.4937 - val_accuracy: 0.7680
Epoch 4/20
100/100 [==============================] - 28s 283ms/step - loss: 0.4015 - accuracy: 0.8409 - val_loss: 0.5601 - val_accuracy: 0.7420
Epoch 5/20
100/100 [==============================] - 24s 238ms/step - loss: 0.3530 - accuracy: 0.8637 - val_loss: 0.4264 - val_accuracy: 0.8410
Epoch 6/20
100/100 [==============================] - 24s 239ms/step - loss: 0.3623 - accuracy: 0.8566 - val_loss: 0.4850 - val_accuracy: 0.7940
Epoch 7/20
100/100 [==============================] - 24s 241ms/step - loss: 0.3450 - accuracy: 0.8722 - val_loss: 0.4394 - val_ac

In [ ]:
model.save('newmodel.h5')

In [ ]:
!unzip -q "/content/drive/MyDrive/Colab Notebooks/inference.zip"

In [ ]:
test = ImageDataGenerator(rescale=1./255).flow_from_directory(directory="/content/test", batch_size=32, target_size=(256,256), class_mode='binary')


Found 6000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import load_model
mymodel=load_model('newmodel.h5')

In [ ]:
mymodel.evaluate(test)

188/188 [==============================] - 16s 83ms/step - loss: 0.1680 - accuracy: 0.9528


[0.16799099743366241, 0.952833354473114]

In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2

In [ ]:
image = cv2.imread("/content/test/Signed/1 (1584).png")

black_img=np.sum(image<=80)
white_img=np.sum(image>80)
percentage=(white_img/black_img)*100
print('black pixels: ',black_img)
print('white_pixels: ',white_img)
print('percentage: ', percentage)

black pixels:  194538
white_pixels:  2070
percentage:  1.0640594639607686


In [ ]:
nosign = cv2.imread("/content/train/Signed/1 (1792).png")
sign = cv2.imread("/content/train/Signed/10 (279).png")

def calc_white_per(img):
  white = np.sum(img != 0)
  black = np.sum(img == 0)
  perc = white/(white+black)
  return perc

print("Percentage of white pixels for sign img : ",calc_white_per(sign))
print("Percentage of white pixels for nosign img : ",calc_white_per(nosign))

Percentage of white pixels for sign img :  1.0
Percentage of white pixels for nosign img :  0.6818389892578125


In [ ]:
sign_dir = "/content/train/Signed"
unsign_dir = "/content/train/Unsigned"

sign_dir_list = os.listdir(sign_dir)
unsign_dir_list = os.listdir(unsign_dir)

sign_dir_list.sort()
unsign_dir_list.sort()

Signedpercentage = 0
for img_nm in sign_dir_list:
    img = cv2.imread(os.path.join(sign_dir,img_nm))
    Signedpercentage += calc_white_per(img)

In [ ]:
Unsignedpercentage = 0
for img_nm in unsign_dir_list:
    img1 = cv2.imread(os.path.join(unsign_dir,img_nm))
    Unsignedpercentage += calc_white_per(img1)

In [ ]:
print(Signedpercentage/len(sign_dir_list))

0.5920936771798019


In [ ]:
print(Unsignedpercentage/len(unsign_dir_list))

0.49537408200152366


In [ ]:
count = 0
for img_nm in sign_dir_list:
    img = cv2.imread(os.path.join(sign_dir,img_nm))
    Signedpercentage = calc_white_per(img)
    if Signedpercentage >=0.59:
      count += 1

In [ ]:
print("For Signed : ")
print("Predicted right : ",count)
print("Predicted wrong : ",len(sign_dir_list)-count)

For Signed : 
Predicted right :  23489
Predicted wrong :  0


In [ ]:
count1 = 0
for img_nm in unsign_dir_list:
    img = cv2.imread(os.path.join(unsign_dir,img_nm))
    Signedpercentage = calc_white_per(img)
    if Signedpercentage <=0.59:
      count1 += 1

In [ ]:
print("For Unsigned : ")
print("Predicted right : ",count1)
print("Predicted wrong : ",len(unsign_dir_list)-count1)

For Unsigned : 
Predicted right :  0
Predicted wrong :  23489


In [ ]:
signed = 0
unsigned = 0
for img_nm in sign_dir_list:
    img = cv2.imread(os.path.join(sign_dir,img_nm))
    Signedpercentage = calc_white_per(img)
    if Signedpercentage >= 0.59:
      signed += 1
    else:
      unsigned += 1

print("No of signed : ",signed)
print("No of unsigned : ",unsigned)

No of signed :  13923
No of unsigned :  9566
